#### Introducción
Aprendizaje supervisado: cada observación tiene una respuesta asociada que guía el aprendizaje.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder 
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
from sklearn import tree


df  = pd.read_csv("data/titanic.csv")

df.info()

#### Entendimiento del problema
Entender el problema en Machine Learning es el primer paso a resolver de todo científico de datos, requiere de habilidades de escucha, comunicación, comprensión y pensamiento crítico. La meta es identificar y expresar con claridad el problema que se no asignó y la solución pedida, para ello necesitamos realizar las preguntas correctas y comunicar de forma clara nuestras deducciones. Este proyecto funciona como un ejemplo básico y didáctico sobre un conjunto de datos cuyo propósito es caracterizar mediante variables cualitativas(nominales y ordinales) y variables cuantitativas(discretas y continuas) a los pasajeros que estuvieron en la tragedia de Titanic, de esta forma es entendible que el objetivo es analizar y describir que ha sucedido para luego averiguar el síntoma de las variables descriptivas sobre las variables clasificadoras que indican la supervivencia de cada pasajero, como análisis final se busca generar un modelo que asocie un resultado a una clasificación en base a la información disponible.

In [4]:
# imputación de datos faltantes mediante medias y modas

df.fillna({"Age":df["Age"].mean()},inplace=True)

mode = df["Embarked"].mode()
df.fillna({"Embarked":mode[0]},inplace=True)

# codificando variables categóricas nominales

df["Sex_encoded"] = LabelEncoder().fit_transform(df["Sex"])

# a su vez generamos una versión textual de la variable binaria objetivo

map_survived = {
    0:"no",
    1:"yes"
}
df["Survived_no_encoded"] = df["Survived"].apply(lambda x : map_survived.get(x))

#### Variable Categóricas y Datasets Desbalanceados
La función de las variables categóricas es brindar información clasificando las observaciones, puenden demostrar una influencia en los datos permitiendonos determinar comportamientos o sacar conclusiones premeditadas si los datos influyen en estas. Los Trip_Priceos de variables categóricas que hay son nominales: aquellas que actúan como etiquetas de los objetos(dentro de estas pueden encontrarse las que se dividen en 2 clases y su codificación las convierte en binarias, es decir, valores posibles de 0 o 1) y ordinales: aquellas que representan un órden jerárquico en particular que rige en las observaciones.

En algunos casos los Datasets tienden a contener más cantidades de datos categóricos que otros, esto puede llegar a traer problemas la hora de entrenar un modelo o evaluar el desempeño de este debido a que la variable minoritaria tiende a ser ignorada y genera un sesgo en cuanto a la importancia.

In [ ]:
survived_count = df["Survived"].value_counts().reset_index()

fig, ax = plt.subplots(1,2, figsize=(10,4))
ax[0].bar(survived_count["Survived_encoded"], survived_count["count"], color=["b","orange"])
ax[0].grid("on")
ax[0].set_title('Conteo de clases')
ax[1].pie(survived_count["count"], labels=survived_count["Survived_encoded"], autopct='%1.1f%%')
ax[1].set_title('Porciones de clase')

plt.subplots_adjust(wspace=0.2, hspace=0.3)
plt.show()

#### CART(classification and regression trees)
Algoritmos basados en calcular umbrales que determinen el resultado de una variable, de manera iterativa se generan particiones sobre la región de información hasta llegar a una formación de grupo lo más homogénea posible. Haciendo analogía a la estructura de un árbol, a partir de la raíz principal se extienden nodos adicionales que representan una condición y hojas que indican la pureza en su región de clases, de esta manera su trayectoria consiste en integorrantes de si o no junto con respuestas asociadas y la fiabilidad de estas. El modelado a partir de esta técnica de aprendizaje requiere controlar la complejidad del árbol de decisión, siendo la pre-poda la acción de limitar el crecimiento durante el proceso de entrenamiento y la post-poda lo mismo luego de este último, esto se realiza mediante la programación de valores númericos definidos y asignados por el usuario, es decir, un ajuste de hiperparámetros y aglunos ejemplos son definir la profundidad máxima o el mínimo número de muestras requeridas para cada partición o hoja.

In [ ]:
# seccionamos los datos en grupos de entrenamiento y prueba

x_train, x_test, y_train, y_test = train_test_split(
                                                   df[["Sex_encoded","Pclass","Fare","Parch","SibSp"]],
                                                   df["Survived"],
                                                   test_size=0.12)

# declaración del algoritmo

tree_decision_clf = tree.DecisionTreeClassifier(criterion="entropy", # medida de aleatoriedad, determinará los nodos del modelo en base a la probabilidad de clasificar erroneamente una variable, pero sin darle mayor importancias a ciertas características
                                           max_depth=6, # distancia entre el nodo(condición) principal y la última hoja(respuesta asociada), valores recomendados: 3-10
                                           min_samples_split=6, # mínimo número de muestras requeridas para dividir un nodo, valores recomendados: 2-10
                                           min_samples_leaf=5, #  mínimo número de muestras requeridas en cada hoja, valores recomendados: 1-10
                                           ccp_alpha=0.003) # valor de post-poda(luego de que el modelo se ajute) que evita el sobreajuste 

model = tree_decision_clf.fit(x_train, y_train)

# predicción de los datos de prueba

class_predicts = model.predict(x_test)

class_real = y_test.values

# Matriz de confusión: recolecta los aciertos y desaciertos tanto de la clase postiva como negativa del modelo

matrix_confusion = confusion_matrix(class_real,class_predicts)
TP = matrix_confusion[0,0]
FP = matrix_confusion[0,1]
FN = matrix_confusion[1,0]
TN = matrix_confusion[1,1]

# Accuracy: indica la acertividad general de nuestro modelo con respecto a las nuevas observaciones

accuracy = accuracy_score(class_real, class_predicts)
color_accuracy = "green"
if accuracy < 0.6:
    color_accuracy = "red"
accuracy_str = str(accuracy)

# Recall: es utilizada para poder saber la efectividad de nuestro modelo a la hora de predecir valores de la clase positiva

recall = recall_score(class_real, class_predicts)
color_recall = "green"
if recall < 0.6:
    color_recall = "red"
recall_str = str(recall)

# Precision: es utilizada para poder saber que porcentage de valores que fueron clasificados como positivos son realmente positivos

precision = precision_score(class_real, class_predicts)
color_precision = "green"
if precision < 0.6:
    color_precision = "red"
precision_str = str(precision)

# F1 Score: es utilizada como un resumen de las dos últimas metricas

F1_score = f1_score(class_real, class_predicts)
color_f1 = "green"
if F1_score < 0.6:
    color_f1 = "red"
F1_score_str = str(F1_score)

print(tree.export_text(model, feature_names=["Sex_encoded","Pclass","Fare","Parch","SibSp"]))

plt.figure(figsize=(35,22))
tree.plot_tree(model, feature_names=["Sex_encoded","Pclass","Fare","Parch","SibSp"])
plt.show()

#### Dashboard que muestra las probabilidades de sobrevivir según las variables y el desempeño del modelo

In [ ]:
# creación de dashboard basado en la estructura HTML/CSS

app = dash.Dash(__name__)

app.layout = html.Div(id="body",className="e1_body",children=[
html.H1("Titanic",id="title",className="e1_title"),
html.Div(className="e1_dashboards",children=[
    html.Div(id="graph_div_1",className="e1_graph_div",children=[
        html.Div(id="dropdown_div_1",className="e1_dropdown_div",children=[
            dcc.Dropdown(id="dropdown_1",className="e1_dropdown",
                        options = [
                            {"label":"Sexo","value":"Sex"},
                            {"label":"Clase social","value":"Pclass"},
                            {"label":"Embarcadero","value":"Embarked"},
                            {"label":"Padres e hijos/as","value":"Parch"},
                            {"label":"Hermanas/os y esposas/os","value":"SibSp"},
                        ],
                        value="Sex",
                        multi=False,
                        clearable=False)
        ]),
        dcc.Graph(id="piechart",className="e1_graph",figure={})
    ]),
    html.Div(id="graph_div_2",className="e1_graph_div",children=[
        html.Div(id="dropdown_div_2",className="e1_dropdown_div",children=[
            dcc.Dropdown(id="dropdown_2",className="e1_dropdown",
                        options = [
                            {"label":"Edad","value":"Age"},
                            {"label":"Boleto","value":"Fare"},
                        ],
                        value="Age",
                        multi=False,
                        clearable=False)
        ]),
        dcc.Graph(id="bar",className="e1_graph",figure={})
    ]),
]),
    
    html.Div(className="e1_div", children=[
        html.Div(id="performance", className="e1_performance",children=[
            html.P([html.B("Clases reales", style={"color":"blue"}),"   |   ",html.B("Predicciones",style={"color":"red"})], style={"text-align":"center","font-family":"sans-serif"}),
            html.P("--------------------------------------------------------------------------------------------------------------------------------------",style={"margin":"0"}),
            html.P(f"{class_real}", className="e1_real_class"),
            html.P(f"{class_predicts}", className="e1_predicts")
        ]),
        html.Div(id="metrics", className="e1_metrics", children=[
                html.P("Matriz de confusión", style={"font-size":"0.9em","text-align":"center","font-family":"sans-serif","font-weigth":"bold"}),
                html.Div(id="matrix", className="e1_matrix", children=[
                html.Div([html.B(TP,style={"color":"green","font-family":"sans-serif"})],id="TP",className="e1_successes"), 
                html.Div([html.B(FP,style={"color":"red","font-family":"sans-serif"})],id="FP",className="e1_mistakes"),
                html.Div([html.B(FN,style={"color":"red","font-family":"sans-serif"})],id="FN",className="e1_mistakes"),
                html.Div([html.B(TN,style={"color":"green","font-family":"sans-serif"})],id="TN",className="e1_successes")
                ]),
                html.Div(id="scores",children=[
                html.Ul(id="list",children=[
                html.Li([f"Accuracy: ",html.B(accuracy_str[:4],style={"color":f"{color_accuracy}"})],id="accuracy",className="e1_score"),
                html.Li([f"Recall: ",html.B(recall_str[:4],style={"color":f"{color_recall}"})],id="recall",className="e1_score"),
                html.Li([f"Precision: ",html.B(precision_str[:4],style={"color":f"{color_precision}"})],id="precision",className="e1_score"),
                html.Li([f"F1 Score: ",html.B(F1_score_str[:4],style={"color":f"{color_f1}"})],id="f1_score",className="e1_score")
                ])
                
            ])
        ])
    ])
])

# generan interacción entre los datos de entrada y los elementos que serán modificados 

@app.callback(
    [Output(component_id="piechart",component_property="figure"),
    Output(component_id="bar",component_property="figure")],
    [Input(component_id="dropdown_1",component_property="value"),
    Input(component_id="dropdown_2",component_property="value")]
)

# función que se ejecuta cada vez que se interactúa con los elementos 

def update_graph(slct_var_cat,slct_var_num):
    
    df_percentage = df.groupby(slct_var_cat)["Survived"].mean().reset_index()
    df_percentage["Survived"] = round(df_percentage["Survived"] * 100)
    df_percentage["Survived"] = df_percentage["Survived"].astype(str)
    df_percentage["var_percentage"] = df_percentage[slct_var_cat].astype(str)+"("+df_percentage["Survived"]+"%)"
    
    piechart = px.pie(df_percentage, values="Survived", names="var_percentage", title="Probabilidad de supervivencia")
    
    df_mean = df.groupby("Survived_no_encoded")[slct_var_num].mean().reset_index()
    
    barplot = px.bar(df_mean, x="Survived_no_encoded", y=slct_var_num, title="Medias de Edad y Boleto",labels={"x":"sobrevivientes","y":slct_var_num})
    barplot.update_layout(xaxis_title="Sobrevivientes")
    
    return piechart,barplot

if __name__ == "__main__":
    app.run_server(debug=False)

#### Error de varianza

Es la variabilidad que existe en la función objetivo con respecto a los diferentes datos de entrenamiento que se utilicen para la creación del modelo, sucede principalmente en algoritmos que se ajustan fácilmente a los datos y requieren menos suposiciones a diferencia de los algoritmos con alto bías, algunos ejemplos de algoritmos con alta variranza son: KNN, CART o SVM.

In [ ]:
accuracies = []
bucle = list(range(14))

for i in bucle:
    x_train, x_test, y_train, y_test = train_test_split(
                                                   df[["Sex_encoded","Pclass","Fare","Parch","SibSp"]],
                                                   df["Survived"],
                                                   test_size=0.25)
    class_real = y_test.values
    class_predicts = model.predict(x_test)
    accuracy = accuracy_score(class_real, class_predicts)
    accuracies.append(accuracy)
    
plt.plot(bucle, accuracies, marker="*", c="r")
plt.grid("on")
plt.title("Error de Varianza")
plt.ylabel("Acuracies")
plt.show()